<h1><strong>GridSearch para Redes Neurais Recorrentes (RNN) na Predição do Mercado Financeiro</strong></h1>
<h4><strong>SOBRE O ARQUIVO</strong></h4>
<p align="justify">&emsp;Este arquivo complementa o projeto "Modelos de Machine Learning Aplicados ao Mercado Financeiro", mais especificamente, ele foi utilizado antes do Arquivo Principal "PredicaoMercadoFinanceiro.ipynb". Neste arquivo, abordamos os possíveis parâmetros e a melhor escolha desses parâmetros para serem utilizadas nos modelos do Arquivo Principal.</p>
<br>
<hr>

<h4><strong>ÍNDICE</strong></h4>
<a href = "#1">1. Tratamento da Base de Dados</a>
<br>
<a href = "#2">2. GridSearch SimpleRNN</a>
<br>
&emsp;<a href = "#2.1">2.1 SimpleRNN - Dataset Cru</a>
<br>
&emsp;<a href = "#2.2">2.2 SimpleRNN - Dataset com Ruído</a>
<br>
<a href = "#3">3. GridSearch LSTM</a>
<br>
&emsp;<a href = "#3.1">3.1 LSTM - Dataset Cru</a>
<br>
&emsp;<a href = "#3.2">3.2 LSTM - Dataset com Ruído</a>
<br><br>
<hr>

<h4><strong>1. TRATAMENTO DA BASE DE DADOS</strong></h4>
<a id="1"></a>

In [ ]:
# -- INTALAÇÃO DE BIBLIOTÉCAS NECESSÁRIAS --
# !pip install yfinance
# !pip install tensorflow
# !pip install pandas
# !pip install matplotlib

In [ ]:
# -- ADIQUIRINDO ACESSO AS INFORMAÇÕES DA AÇÃO --
# Lista de tickets de ações a serem utilizadas
acao = "VALE3.SA"

# Adquirindo e tratando os valores das ações
import yfinance as yf
data = yf.download( tickers = acao, period = "8d", interval = "1m", auto_adjust = True, ignore_tz = True )
data.dropna( inplace = True ) # removendo valores nulos
display( data[ "Close" ] )
dt = data.index # salvando as datas e o tempo para visualização
data = data[ "Close" ].values # mantendo apenas os valores dos dados de fechamento (o valor no momento em que o registro foi feito)

# Criando as variáções do dataset
import random as rnd
import numpy as np
import copy
data_raw = data[:]
data_noise = list()
for i in data_raw:
  data_noise.append(i + rnd.gauss(0, 0.1))
data_noise = np.array( data_noise )

In [ ]:
# Exibição dos Diferentes Dataset's
janela_visualizacao = len(data_raw)
tick_indices = range( 0, len( dt[ -janela_visualizacao - 1 : ] ), 200)
tick_labels = list( dt[ -janela_visualizacao - 1 : : 200] )

import matplotlib.pyplot as plt
plt.rcParams[ "figure.figsize" ] = ( 12, 4 )
plt.plot( data_noise, label = 'Dataset com Ruído', alpha = 0.8, color = 'yellowgreen' )
plt.plot( data_raw, label = 'Dataset Cru', alpha = 0.8, color = 'firebrick' )
plt.title( f"DATASET's", fontsize = 14 )
plt.xlabel( "Tempo" )
plt.xticks( tick_indices, tick_labels, rotation = 45 )
plt.ylabel( "Preço (R$)" )
plt.legend()
plt.grid( True )
plt.show()

In [ ]:
# -- TRATANDO E DIVIDINDO OS DADOS DA AÇÃO --
# Função para a criação de janelas deslizantes
def create_dataset( series, janela = 10 ):
  X, y = [], []
  for i in range( 0, len( series ) - janela ):
    X.append( series[ i : i + janela ] )
    y.append( series[ i + janela ] )
  return np.array( X ), np.array( y )

# Inicialização do scaler para a normalização
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler( feature_range = ( 0, 1 ) )

train_size = int( 0.8 * len( data_raw ) )
# DATASET CRU
# Divisão treino/teste
train_data_raw, test_data_raw = data_raw[ : train_size ], data_raw[ train_size : ]
# Normalização dos dados
train_data_raw = scaler.fit_transform( train_data_raw.reshape( -1, 1 ) )
test_data_raw = scaler.transform( test_data_raw.reshape( -1, 1 ) )

# DATASET COM RUÍDO
# Divisão treino/teste
train_data_noise, test_data_noise = data_noise[ : train_size ], data_noise[ train_size : ]
# Normalização dos dados
train_data_noise = scaler.fit_transform( train_data_noise.reshape( -1, 1 ) )
test_data_noise = scaler.transform( test_data_noise.reshape( -1, 1 ) )

<br><hr>

<h4><strong>2. GRIDSEARCH SIMPLE_RNN</strong></h4>
<a id="2"></a>

In [ ]:
# -- OPÇÕES GLOBAIS --
num_layers_options = [1, 2]
window_size_options = [10, 20, 30, 40, 50]
neurons_options = [32, 64, 128, 256]
activation_options = ['relu', 'tanh', 'sigmoid']
batch_size_options = [16, 32, 64]

<h5><strong>2.1 Dataset Cru</strong></h5>
<a id = "2.1"></a>

In [ ]:
# -- GRIDSEARCH PARA O DATASET CRU --
# Lista para armazenar os resultados
results_raw_SimpleRNN = list()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.metrics import mean_squared_error
from itertools import product

for num_layers in num_layers_options:
  if num_layers == 1:
    for ws, h, act, bs in product( window_size_options, neurons_options, activation_options, batch_size_options ):
      print( f"Testando: num_layers = {num_layers}, janela = {ws}, hidden = {h}, activation = {act}, batch_size = {bs}" )
      # Criação das janelas
      X_test, y_test = create_dataset( test_data_raw, ws )
      X_train_raw, y_train_raw = create_dataset( train_data_raw, ws )

      # Crição do modelo
      model_raw_SimpleRNN = Sequential()
      model_raw_SimpleRNN.add( SimpleRNN( units = h, activation = act, input_shape = ( ws, 1 ) ) )
      model_raw_SimpleRNN.add( Dense( units = 1 ) )
      model_raw_SimpleRNN.compile( optimizer = "adam", loss = "mean_squared_error" )

      # Treinamento do modelo
      model_raw_SimpleRNN.fit( X_train_raw, y_train_raw, epochs = 100, batch_size = bs, verbose = 0 )

      # Salvando o resultado
      y_real = scaler.inverse_transform( y_test )
      y_pred_raw = scaler.inverse_transform( model_raw_SimpleRNN.predict( X_test ) )
      mse = mean_squared_error( y_real, y_pred_raw )
      results_raw_SimpleRNN.append( ( ( num_layers, ws, h, act, bs ), mse ) )
  else:
    for ws, h1, h2, act, bs in product( window_size_options, neurons_options, neurons_options, activation_options, batch_size_options ):
      print( f"Testando: num_layers = {num_layers}, janela = {ws}, hidden1 = {h1}, hidden2 = {h2}, activation = {act}, batch_size = {bs}" )
      # Criação das janelas
      X_test, y_test = create_dataset( test_data_raw, ws )
      X_train_raw, y_train_raw = create_dataset( train_data_raw, ws )

      # Crição do modelo
      model_raw_SimpleRNN = Sequential()
      model_raw_SimpleRNN.add( SimpleRNN( units = h1, activation = act, return_sequences = True, input_shape = ( ws, 1 ) ) )
      model_raw_SimpleRNN.add( SimpleRNN( units = h2, activation = act ) )
      model_raw_SimpleRNN.add( Dense( units = 1 ) )
      model_raw_SimpleRNN.compile( optimizer = "adam", loss = "mean_squared_error" )

      # Treinamento do modelo
      model_raw_SimpleRNN.fit( X_train_raw, y_train_raw, epochs = 100, batch_size = bs, verbose = 0 )

      # Salvando o resultado
      y_real = scaler.inverse_transform( y_test )
      y_pred_raw = scaler.inverse_transform( model_raw_SimpleRNN.predict( X_test ) )
      mse = mean_squared_error( y_real, y_pred_raw )
      results_raw_SimpleRNN.append( ( ( num_layers, ws, h1, h2, act, bs ), mse ) )

In [ ]:
# -- OBTER O MELHOR MODELO DO DATASET CRU --
best_model_raw_SimpleRNN = min( results_raw_SimpleRNN, key=lambda x: x[ 1 ] )
if best_model_raw_SimpleRNN[ 0 ][ 0 ] == 1:
  num_layers, ws, h, act, bs = best_model_raw_SimpleRNN[ 0 ]
  print( f"Melhor modelo: num_layers = {num_layers}, janela = {ws}, hidden = {h}, activation = {act}, batch_size = {bs}" )
else:
  num_layers, ws, h1, h2, act, bs = best_model_raw_SimpleRNN[ 0 ]
  print( f"Melhor modelo: num_layers = {num_layers}, janela = {ws}, hidden1 = {h1}, hidden2 = {h2}, activation = {act}, batch_size = {bs}" )

<h5><strong>2.2 Dataset com Ruído</strong></h5>
<a id = "2.2"></a>

In [ ]:
# -- GRIDSEARCH PARA O DATASET COM RUÍDO --
# Lista para armazenar os resultados
results_noise_SimpleRNN = list()
for num_layers in num_layers_options:
  if num_layers == 1:
    for ws, h, act, bs in product( window_size_options, neurons_options, activation_options, batch_size_options ):
      print( f"Testando: num_layers = {num_layers}, janela = {ws}, hidden = {h}, activation = {act}, batch_size = {bs}" )
      # Criação das janelas
      X_test, y_test = create_dataset( test_data_raw, ws )
      X_train_noise, y_train_noise = create_dataset( train_data_noise, ws )

      # Crição do modelo
      model_noise_SimpleRNN = Sequential()
      model_noise_SimpleRNN.add( SimpleRNN( units = h, activation = act, input_shape = ( ws, 1 ) ) )
      model_noise_SimpleRNN.add( Dense( units = 1 ) )
      model_noise_SimpleRNN.compile( optimizer = "adam", loss = "mean_squared_error" )

      # Treinamento do modelo
      model_noise_SimpleRNN.fit( X_train_noise, y_train_noise, epochs = 100, batch_size = bs, verbose = 0 )

      # Salvando o resultado
      y_real = scaler.inverse_transform( y_test )
      y_pred_noise = scaler.inverse_transform( model_noise_SimpleRNN.predict( X_test ) )
      mse = mean_squared_error( y_real, y_pred_noise )
      results_noise_SimpleRNN.append( ( ( num_layers, ws, h, act, bs ), mse ) )
  else:
    for ws, h1, h2, act, bs in product( window_size_options, neurons_options, neurons_options, activation_options, batch_size_options ):
      print( f"Testando: num_layers = {num_layers}, janela = {ws}, hidden1 = {h1}, hidden2 = {h2}, activation = {act}, batch_size = {bs}" )
      # Criação das janelas
      X_test, y_test = create_dataset( test_data_raw, ws )
      X_train_noise, y_train_noise = create_dataset( train_data_noise, ws )

      # Crição do modelo
      model_noise_SimpleRNN = Sequential()
      model_noise_SimpleRNN.add( SimpleRNN( units = h1, activation = act, return_sequences = True, input_shape = ( ws, 1 ) ) )
      model_noise_SimpleRNN.add( SimpleRNN( units = h2, activation = act ) )
      model_noise_SimpleRNN.add( Dense( units = 1 ) )
      model_noise_SimpleRNN.compile( optimizer = "adam", loss = "mean_squared_error" )

      # Treinamento do modelo
      model_noise_SimpleRNN.fit( X_train_noise, y_train_noise, epochs = 100, batch_size = bs, verbose = 0 )

      # Salvando o resultado
      y_real = scaler.inverse_transform( y_test )
      y_pred_noise = scaler.inverse_transform( model_noise_SimpleRNN.predict( X_test ) )
      mse = mean_squared_error( y_real, y_pred_noise )
      results_noise_SimpleRNN.append( ( ( num_layers, ws, h1, h2, act, bs ), mse ) )

In [ ]:
# -- OBTER O MELHOR MODELO DO DATASET COM RUÍDO --
best_model_noise_SimpleRNN = min( results_noise_SimpleRNN, key=lambda x: x[ 1 ] )
if best_model_noise_SimpleRNN[ 0 ][ 0 ] == 1:
  num_layers, ws, h, act, bs = best_model_noise_SimpleRNN[ 0 ]
  print( f"Melhor modelo: num_layers = {num_layers}, janela = {ws}, hidden = {h}, activation = {act}, batch_size = {bs}" )
else:
  num_layers, ws, h1, h2, act, bs = best_model_noise_SimpleRNN[ 0 ]
  print( f"Melhor modelo: num_layers = {num_layers}, janela = {ws}, hidden1 = {h1}, hidden2 = {h2}, activation = {act}, batch_size = {bs}" )

<br><hr>

<h4><strong>3. GRIDSEARCH LSTM</strong></h4>
<a id = "3"></a>

In [ ]:
# -- OPÇÕES GLOBAIS -- (As mesmas opções que a do SimpleRNN )
# num_layers_options = [1, 2]
# window_size_options = [10, 20, 30, 40, 50]
# neurons_options = [32, 64, 128, 256]
# activation_options = ['relu', 'tanh', 'sigmoid']
# batch_size_options = [16, 32, 64]

<h5><strong>3.1 Dataset Cru</strong></h5>
<a id = "3.1"></a>

In [ ]:
# -- GRIDSEARCH PARA O DATASET CRU --
# Lista para armazenar os resultados
results_raw_LSTM = list()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
from itertools import product
for num_layers in num_layers_options:
  if num_layers == 1:
    for ws, h, act, bs in product( window_size_options, neurons_options, activation_options, batch_size_options ):
      print( f"Testando: num_layers = {num_layers}, janela = {ws}, hidden = {h}, activation = {act}, batch_size = {bs}" )
      # Criação das janelas
      X_test, y_test = create_dataset( test_data_raw, ws )
      X_train_raw, y_train_raw = create_dataset( train_data_raw, ws )

      # Crição do modelo
      model_raw_LSTM = Sequential()
      model_raw_LSTM.add( LSTM( units = h, activation = act, input_shape = ( ws, 1 ) ) )
      model_raw_LSTM.add( Dense( units = 1 ) )
      model_raw_LSTM.compile( optimizer = "adam", loss = "mean_squared_error" )

      # Treinamento do modelo
      model_raw_LSTM.fit( X_train_raw, y_train_raw, epochs = 100, batch_size = bs, verbose = 0 )

      # Salvando o resultado
      y_real = scaler.inverse_transform( y_test )
      y_pred_raw = scaler.inverse_transform( model_raw_LSTM.predict( X_test ) )
      mse = mean_squared_error( y_real, y_pred_raw )
      results_raw_LSTM.append( ( ( num_layers, ws, h, act, bs ), mse ) )
  else:
    for ws, h1, h2, act, bs in product( window_size_options, neurons_options, neurons_options, activation_options, batch_size_options ):
      print( f"Testando: num_layers = {num_layers}, janela = {ws}, hidden1 = {h1}, hidden2 = {h2}, activation = {act}, batch_size = {bs}" )
      # Criação das janelas
      X_test, y_test = create_dataset( test_data_raw, ws )
      X_train_raw, y_train_raw = create_dataset( train_data_raw, ws )

      # Crição do modelo
      model_raw_LSTM = Sequential()
      model_raw_LSTM.add( LSTM( units = h1, activation = act, return_sequences = True, input_shape = ( ws, 1 ) ) )
      model_raw_LSTM.add( LSTM( units = h2, activation = act ) )
      model_raw_LSTM.add( Dense( units = 1 ) )
      model_raw_LSTM.compile( optimizer = "adam", loss = "mean_squared_error" )

      # Treinamento do modelo
      model_raw_LSTM.fit( X_train_raw, y_train_raw, epochs = 100, batch_size = bs, verbose = 0 )

      # Salvando o resultado
      y_real = scaler.inverse_transform( y_test )
      y_pred_raw = scaler.inverse_transform( model_raw_LSTM.predict( X_test ) )
      mse = mean_squared_error( y_real, y_pred_raw )
      results_raw_LSTM.append( ( ( num_layers, ws, h1, h2, act, bs ), mse ) )

In [ ]:
# -- OBTER O MELHOR MODELO DO DATASET CRU --
best_model_raw_LSTM = min( results_raw_LSTM, key=lambda x: x[ 1 ] )
if best_model_raw_LSTM[ 0 ][ 0 ] == 1:
  num_layers, ws, h, act, bs = best_model_raw_LSTM[ 0 ]
  print( f"Melhor modelo: num_layers = {num_layers}, janela = {ws}, hidden = {h}, activation = {act}, batch_size = {bs}" )
else:
  num_layers, ws, h1, h2, act, bs = best_model_raw_LSTM[ 0 ]
  print( f"Melhor modelo: num_layers = {num_layers}, janela = {ws}, hidden1 = {h1}, hidden2 = {h2}, activation = {act}, batch_size = {bs}" )

<h5><strong>3.2 Dataset com Ruído</strong></h5>
<a id = "3.2"></a>

In [ ]:
# -- GRIDSEARCH PARA O DATASET COM RUÍDO --
# Lista para armazenar os resultados
results_noise_LSTM = list()
for num_layers in num_layers_options:
  if num_layers == 1:
    for ws, h, act, bs in product( window_size_options, neurons_options, activation_options, batch_size_options ):
      print( f"Testando: num_layers = {num_layers}, janela = {ws}, hidden = {h}, activation = {act}, batch_size = {bs}" )
      # Criação das janelas
      X_test, y_test = create_dataset( test_data_raw, ws )
      X_train_noise, y_train_noise = create_dataset( train_data_noise, ws )

      # Crição do modelo
      model_noise_LSTM = Sequential()
      model_noise_LSTM.add( LSTM( units = h, activation = act, input_shape = ( ws, 1 ) ) )
      model_noise_LSTM.add( Dense( units = 1 ) )
      model_noise_LSTM.compile( optimizer = "adam", loss = "mean_squared_error" )

      # Treinamento do modelo
      model_noise_LSTM.fit( X_train_noise, y_train_noise, epochs = 100, batch_size = bs, verbose = 0 )

      # Salvando o resultado
      y_real = scaler.inverse_transform( y_test )
      y_pred_noise = scaler.inverse_transform( model_noise_LSTM.predict( X_test ) )
      mse = mean_squared_error( y_real, y_pred_noise )
      results_noise_LSTM.append( ( ( num_layers, ws, h, act, bs ), mse ) )
  else:
    for ws, h1, h2, act, bs in product( window_size_options, neurons_options, neurons_options, activation_options, batch_size_options ):
      print( f"Testando: num_layers = {num_layers}, janela = {ws}, hidden1 = {h1}, hidden2 = {h2}, activation = {act}, batch_size = {bs}" )
      # Criação das janelas
      X_test, y_test = create_dataset( test_data_raw, ws )
      X_train_noise, y_train_noise = create_dataset( train_data_noise, ws )

      # Crição do modelo
      model_noise_LSTM = Sequential()
      model_noise_LSTM.add( LSTM( units = h1, activation = act, return_sequences = True, input_shape = ( ws, 1 ) ) )
      model_noise_LSTM.add( LSTM( units = h2, activation = act ) )
      model_noise_LSTM.add( Dense( units = 1 ) )
      model_noise_LSTM.compile( optimizer = "adam", loss = "mean_squared_error" )

      # Treinamento do modelo
      model_noise_LSTM.fit( X_train_noise, y_train_noise, epochs = 100, batch_size = bs, verbose = 0 )

      # Salvando o resultado
      y_real = scaler.inverse_transform( y_test )
      y_pred_noise = scaler.inverse_transform( model_noise_LSTM.predict( X_test ) )
      mse = mean_squared_error( y_real, y_pred_noise )
      results_noise_LSTM.append( ( ( num_layers, ws, h1, h2, act, bs ), mse ) )

In [ ]:
# -- OBTER O MELHOR MODELO DO DATASET CRU --
best_model_noise_LSTM = min( results_noise_LSTM, key=lambda x: x[ 1 ] )
if best_model_noise_LSTM[ 0 ][ 0 ] == 1:
  num_layers, ws, h, act, bs = best_model_noise_LSTM[ 0 ]
  print( f"Melhor modelo: num_layers = {num_layers}, janela = {ws}, hidden = {h}, activation = {act}, batch_size = {bs}" )
else:
  num_layers, ws, h1, h2, act, bs = best_model_noise_LSTM[ 0 ]
  print( f"Melhor modelo: num_layers = {num_layers}, janela = {ws}, hidden1 = {h1}, hidden2 = {h2}, activation = {act}, batch_size = {bs}" )